In [1]:
from style_transfer import StyleTransfer

In [2]:
from style_transfer import srgb_profile

In [3]:
import argparse
import atexit
from dataclasses import asdict
import io
import json
from pathlib import Path
import platform
import sys
import webbrowser

import numpy as np
from PIL import Image, ImageCms
from tifffile import TIFF, TiffWriter
import torch
import torch.multiprocessing as mp
from tqdm import tqdm

In [4]:
def print_error(err):
    print('\033[31m{}:\033[0m {}'.format(type(err).__name__, err), file=sys.stderr)

In [5]:
def prof_to_prof(image, src_prof, dst_prof, **kwargs):
    src_prof = io.BytesIO(src_prof)
    dst_prof = io.BytesIO(dst_prof)
    return ImageCms.profileToProfile(image, src_prof, dst_prof, **kwargs)


def load_image(path, proof_prof=None):
    src_prof = dst_prof = srgb_profile
    try:
        image = Image.open(path)
        if 'icc_profile' in image.info:
            src_prof = image.info['icc_profile']
        else:
            image = image.convert('RGB')
        if proof_prof is None:
            if src_prof == dst_prof:
                return image.convert('RGB')
            return prof_to_prof(image, src_prof, dst_prof, outputMode='RGB')
        proof_prof = Path(proof_prof).read_bytes()
        cmyk = prof_to_prof(image, src_prof, proof_prof, outputMode='CMYK')
        return prof_to_prof(cmyk, proof_prof, dst_prof, outputMode='RGB')
    except OSError as err:
        print_error(err)
        sys.exit(1)


In [20]:
image1="style_transfer\khalid2.jpeg"
image2="style_transfer\iayoub1.jpeg"

In [21]:
content_img = load_image(image1)


In [22]:
style_imgs = load_image(image2)

In [31]:
!python style_transfer/style_transfer.py

In [35]:
try:
    StyleTransfer().stylize(content_image=content_img, style_images=[style_imgs])
except KeyboardInterrupt:
    pass


Processing content image (128x122)...
Processing style image (119x128)...
Processing content image (181x172)...
Processing style image (168x181)...
Processing content image (256x244)...
Processing style image (238x256)...
